# CICERO -- NRMS Small Dataset Adapted to SUBER first Steps

This notebook is a run of the [NRMS notebook]( adapted from the [Recommenders Team](https://github.com/recommenders-team/recommenders/tree/main).  It has been modified as time has caused some things to not work as presneted originally.

NRMS stands for `Neural News Reccomendaiton with Multi-head Self-Attention`.  The reference to the paper is provided below. please look over and study the [recommenders team github repository](https://github.com/recommenders-team/recommenders/blob/main/README.md#Getting-Started) because this code is derived from it. 

Read up on the [MIND: MIcrosoft News Dataset](https://msnews.github.io/) and download the data into the datasets folder of this project and place them in `/apps/datasets`.  Review the [Readme](/app/datasets/README.md) 

## do imports and check things out.

Are we using a GPU? If not then things will take a very long time.


In [ ]:
import time

# Start the timer
start_time = time.time()

# Remove warnings
import os
os.environ['TF_TRT_ALLOW_ENGINE_NATIVE_SEGMENT_EXECUTION'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
# Gets the SUBERX python modules included.
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import numpy as np
import pandas as pd
import pickle
import zipfile
import random
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

# This module was created to generate the necessary files if they dont' already exist
from environment.data_utils import generate_uid2index, load_glove_embeddings, create_word_dict_and_embeddings, setup_nltk_resources

from nltk.tokenize import word_tokenize
import nltk
from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.datasets.mind import download_and_extract_glove
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

# List available devices
print("Available devices:")
for device in tf.config.list_physical_devices():
    print(device)

# Check if a GPU is detected
if tf.config.list_physical_devices('GPU'):
    print("GPU is available and TensorFlow is using it.")
else:
    print("GPU is NOT available. TensorFlow is using the CPU.")


## Prepare Parameters

Adjust these as needed. 



In [ ]:
epochs = 5
seed = 42
batch_size = 64


# My modification for MINDSMALL as I mount in the data into the container.  Further the small dataset is not accessable anymore.

# Options: MINDdemo, MINDsmall, MINDlarge

MIND_type = 'MINDsmall'

## Specify the dataset to use

There are three in the original notebook: `demo`, `small` and `large`.

From the [original  notebook](https://github.com/recommenders-team/recommenders/blob/main/examples/00_quick_start/nrms_MIND.ipynb),  the `demo` set is 5000 samples of the `small` dataset.

I was able to download the original demo but that's not really needed. A sample of 5000 is good for showing the algorithm works but won't train it well enough.  

In [ ]:
## I mount in the datasets folder to /app
data_path_base="/app/datasets/"
data_path = data_path_base + MIND_type

# Create the directory if it doesn't exist
os.makedirs(data_path, exist_ok=True)

print(f"Data Path is {data_path}")

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
dev_news_file = os.path.join(data_path,'dev','news.tsv')
dev_behaviors_file = os.path.join(data_path,'dev','behaviors.tsv')

wordEmb_file = os.path.join(data_path, 'utils',"embedding.npy")
userDict_file = os.path.join(data_path, 'utils',"uid2index.pkl")
wordDict_file = os.path.join(data_path, 'utils',"word_dict.pkl")
yaml_file = os.path.join(data_path, "utils",'nrms.yaml')


## Download Glove embeddings

The original NRMS used glove embeddings, this is how they are created. 

In [ ]:
## All models can use the same glove embeddings

# Download and extract GloVe embeddings
glove_dir = data_path_base +'glove_embeddings'
download_and_extract_glove(glove_dir)
print(f"GloVe embeddings extracted to: {glove_dir}")




## Get the NLTK Data

In [ ]:
nltk_data_dir = data_path_base + 'nltk_data'
setup_nltk_resources(download_dir=nltk_data_dir)
nltk.data.path.append(nltk_data_dir)

## Identify the news and behavior files

In [ ]:
# Generate uid2index.pkl
uid2index = generate_uid2index(train_behaviors_file, output_file= userDict_file)

In [ ]:
# Path to GloVe file (e.g., glove.6B.300d.txt)
glove_file = os.path.join(glove_dir, "glove/glove.6B.300d.txt")
embedding_dim = 300
glove_embeddings = load_glove_embeddings(glove_file, embedding_dim=embedding_dim)


In [ ]:
# Generate word_dict.pkl and embedding.npy
word_dict, embedding_matrix = create_word_dict_and_embeddings(
    news_file= train_news_file,
    glove_embeddings=glove_embeddings,
    embedding_dim=embedding_dim,
    output_dir= data_path + '/utils/'
)


## Define the hyper-parameters

In [ ]:
hparams = prepare_hparams(None, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          model_type="nrms",
                          title_size=30,
                          his_size=50,
                          npratio=4,
                          data_format='news',
                          word_emb_dim=300,
                          head_num=20,
                          head_dim=20,
                          attention_hidden_dim=200,
                          loss='cross_entropy_loss',
                          dropout=0.2,
                          support_quick_scoring=True,
                          show_step=10, 
                          metrics=['group_auc', 'mean_mrr', 'ndcg@5;10'])
#print(hparams)

In [ ]:
for i in hparams.values().keys():
    print(f"{i} = {hparams.values()[i]}")

In [ ]:
iterator = MINDIterator
model = NRMSModel(hparams, iterator, seed=seed)

## Figuring out for SUBER Input

1. `parsed_data` is a generator.
2. `data_d` is a dictionary of one batch of size 64 each.
3. Showing the keys shows the 64 batch size. 

In [ ]:
parsed_data = model.train_iterator.load_data_from_file(train_news_file, train_behaviors_file)
print(type(parsed_data))
data_d = next(parsed_data)
print(type(data_d))

for key, value in data_d.items():
    print(f"Key: {key}, Shape: {value.shape if hasattr(value, 'shape') else type(value)}")



### Description of keys
*Note the first dimension is 64 for all keys.*
- `impression_index_batch`(64,1): comes out of the behaviors tsv, one impression id per user in the batch.
- `user_index_batch` (64,1): One user id per entry.
- `clicked_title_batch`: (64,50,30) Each user has a click history of up to 50 articles. each article title has 30 words.
- `candidate_tile_batch` (64,5,30): Each user is shown 5 candidate articles. Each article title has 30 words.
-  `labels` (64,5): One label per candidate article foe each user in the batch.This indicates where the user clicked each candidate article (1 for click and 0 for not click).

---

Next, we run a few opochs for `show and tell` -- just to see that the format of our data is correct for input.  We do this by passing it to `model.train` located in `https://github.com/recommenders-team/recommenders/blob/main/recommenders/models/newsrec/models/base_model.py#L150 `. Take a look at `recommenders/models/newsrec/models/nrms.py` particularly the method `_build_nrms`; this is what "builds the model. 

In [ ]:
# An example that illustrates the shape of the input to NRMS.

losses = []
for i, batch in enumerate(parsed_data):
    loss = model.train(batch)
    print(f"Batch {i + 1} Loss: {loss}")
    losses.append(loss)
    if i == 10:
        break

## Mid-Notebook checkpoint

OK, We know how the model will take data in and we know the structure of the data so how can we create it.  

# SUBER Integration of NRMS Dataset

To integrate the synthtic analysts we need to:
1. Do some imports and setup for Pydantic AI
2. Open our synthetic users file.
3. Generate a UID similar to the ones in `behaviors.tsv`.
4. 

In [ ]:
# Imports for Pydantic AI
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.ollama import OllamaModel
from typing import Dict, Optional, List

base_url = "http://ollama:11434/v1/"
# Define the Ollama model running on Ollama
ollama_model = OllamaModel(
    model_name="mistral:7b",  # Replace with your preferred model  Could be 'mistrel:7b', 'granite3.1-dense:latest', 'llama3.2', gemma2
    base_url="http://ollama:11434/v1/"  # Ollama's default base URL
)



In [ ]:
existing_syn_analysts
file_path = os.path.join(data_path_base, "synthetic_analysts.csv")
# Check if the file exists

if os.path.exists(file_path):
    existing_syn_analysts = pd.read_csv(file_path)
    print(f"Existing data loaded. {len(existing_syn_analysts)} records found.")

In [ ]:
# Step 1: Create obfuscated IDs
num_analysts = len(existing_syn_analysts)
obfuscated_ids = [f"A{random.randint(1000, 9999)}" for _ in range(num_analysts)]

zipped_data = list(zip(obfuscated_ids, existing_syn_analysts["name"]))
uid_name_df = pd.DataFrame(zipped_data, columns=["uid", "name"])

In [ ]:
uid_name_df.head()

## Reminder of the behaviors.tsv columns

- `imporession_id`: Basically a unique Identifier for this observation or impression session
- `user_id`       : The unique Identifier of the User or Analysts that saw this record
- `time`          : data time stamp formatted like: `11/11/2019 11:14:32 AM`
- `history`       : a list of articles clicked in the past
- `impressions`   : articles shown in this session with a `1` for clicked or a `0` for did not click.  For this session.

## Reminder of the news.tsv columns

- `news_id`          : A unique ID for each news article
- `category`         : The high-level catagory of the news article.
- `subcategory`      : More granular classification under the category.
- `title`            : The headline of the article.
- `abstract`         : A brief summary or preview of the aricle's content.
- `url`              : Link to the full article (not useful anymore as these are no long valid URLs -- too old)
- `title_entities`   : Structured data indicationg named entities found in the title.
- `abstract_entities`: Structured data indicatin named entities found in the abstract.


In [ ]:
# Load the news.tsv file
news_df = pd.read_csv(train_news_file, sep="\t", header=None)

# Assign column names based on the standard structure of the news.tsv file
news_df.columns = [
    "news_id", 
    "category", 
    "subcategory", 
    "title", 
    "abstract", 
    "url", 
    "title_entities", 
    "abstract_entities"
]

In [ ]:
news_df.head()

## Define a behavior class for Synthetic Users


In [ ]:
from pydantic import BaseModel, Field

class AnalystBehavior(BaseModel):
    """
    This is the structure the LLM will return for behaviors.
    """
    impression_id: str = Field(description="A unique session identifier for this behavior entry.")
    user_id: str = Field(description="The unique identifier of the analyst.")
    time: str = Field(description="Timestamp of the session in MM/DD/YYYY HH:MM:SS AM/PM format.")
    history: str = Field(description="A string of previously clicked article IDs, separated by spaces. Only add the newsID if the article was selected.")
    impressions: str = Field(description=(
        "A string of article IDs separated by spaces, where each ID is followed by '-1' if clicked "
        "or '-0' if not clicked (e.g., 'N12345-1 N23456-0')."
    ))

    def __str__(self):
        return (
            f"AnalystBehavior:\n"
            f"  Impression ID: {self.impression_id}\n"
            f"  User ID: {self.user_id}\n"
            f"  Time: {self.time}\n"
            f"  History: {self.history}\n"
            f"  Impressions: {self.impressions}\n"
        )
    
    def __repr__(self):
        return self.__str__()


## Define an Agent First



In [ ]:
# Create the agent
agent = Agent(model=ollama_model, result_type=AnalystBehavior, retries=5)

## Let's regroup again

We have

1. uid_name_df: The UserID to Name Index
2. existing_syn_analyst
3. train_news_df

And we need these arguments to our function for generating AnalystBehaviors:

1. An Agent
2. An Analyst
3. Histories
4. Impresisons
5. retries
6. delay


In [ ]:
# Join on the "name" column
merged_df = uid_name_df.merge(existing_syn_analysts, on="name", how="inner")
# Convert each analyst row into a dictionary
analyst_dicts = merged_df.to_dict(orient="records")

## Create a History DataFrame and functions. 

This will be saved in datasets -- keep that in mind. 



In [ ]:
# Initialize the history DataFrame
history_df = pd.DataFrame({
    "uid": merged_df["uid"],  # Take UIDs from the merged DataFrame
    "history": [[] for _ in range(len(merged_df))]  # Start with empty lists
})

# Inspect the initial history DataFrame
print("Initial History DataFrame:")
display(history_df.head())


### Function to update and get the history for the UUID

In [ ]:
def update_history(uid, new_articles):
    """
    Append new articles to an analyst's history.
    
    Args:
        uid (str): The UID of the analyst.
        new_articles (list): List of new article IDs to add to history.
    """
    global history_df
    # Locate the analyst's row and update the history
    idx = history_df.index[history_df["uid"] == uid]
    if not idx.empty:
        history_df.at[idx[0], "history"] += new_articles
    else:
        print(f"UID {uid} not found in history_df.")

def get_history(uid):
    """
    Retrieve the history for a specific UID.
    
    Args:
        uid (str): The UID of the analyst.
    
    Returns:
        list: The history of article IDs.
    """
    row = history_df.loc[history_df["uid"] == uid]
    if not row.empty:
        return row.iloc[0]["history"]
    else:
        return []


## Impressions

In [ ]:
# Initialize an empty impressions DataFrame
impressions_df = pd.DataFrame(columns=["impression_id", "uid", "news_id", "clicked"])

# Inspect the empty DataFrame
#print("Initial Impressions DataFrame:")
#print(impressions_df)


In [ ]:
def add_impressions(impression_id, uid, articles, clicks):
    """
    Add impressions for a specific session to the impressions DataFrame.
    
    Args:
        impression_id (str): Unique ID for the session.
        uid (str): Analyst's unique ID.
        articles (list): List of news article IDs shown in the session.
        clicks (list): List of binary values indicating if each article was clicked (1) or not clicked (0).
    """
    global impressions_df
    new_impressions = pd.DataFrame({
        "impression_id": [impression_id] * len(articles),
        "uid": [uid] * len(articles),
        "news_id": articles,
        "clicked": clicks
    })
    impressions_df = pd.concat([impressions_df, new_impressions], ignore_index=True)

# Example: Add impressions for a session
#add_impressions("IMP001", "A1234", ["N1", "N2", "N3"], [1, 0, 1])

def get_impressions(impression_id=None, uid=None):
    """
    Retrieve impressions for a specific session or analyst.
    
    Args:
        impression_id (str): The session ID to filter by (optional).
        uid (str): The UID of the analyst to filter by (optional).
    
    Returns:
        pd.DataFrame: A filtered DataFrame of impressions.
    """
    if impression_id:
        return impressions_df[impressions_df["impression_id"] == impression_id]
    elif uid:
        return impressions_df[impressions_df["uid"] == uid]
    else:
        return impressions_df  # Return all impressions if no filters are applied


## 1. Select a Session

Sessions are what is shown to the analyst. Behaviors are what the analyst did with what was shown.

In [ ]:
def select_articles_for_session(news_df, num_articles=5):
    """
    Select a fixed number of articles randomly from the news DataFrame.
    
    Args:
        news_df (pd.DataFrame): The DataFrame containing news articles.
        num_articles (int): The number of articles to select for the session.

    Returns:
        pd.DataFrame: A DataFrame of selected articles for the session.
    """
    return news_df.sample(n=num_articles)

# Example usage: Select 5 articles
#session_articles = select_articles_for_session(news_df, num_articles=5)

## An example for testing

In [ ]:
selected_uid = random.choice(history_df["uid"].tolist())
selected_uid = 'A3020'
# TODO -- may be want to distribute evenly -- make sure it does.
analyst = merged_df.loc[merged_df["uid"] == selected_uid].to_dict(orient="records")[0]
history = get_history(selected_uid)
behaviors_df = pd.read_csv(train_behaviors_file, sep="\t", header=None)
behaviors_df.columns = ["impression_id", "user_id", "time", "history", "impressions"]
max_impression_id = behaviors_df["impression_id"].max()
impression_id = max_impression_id + 1

train_news_df = pd.read_csv(train_news_file, sep="\t", header=None)
train_news_df.columns = [
    "news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"
]


session_articles = select_articles_for_session(train_news_df)

impressions = session_articles[["news_id","title"]].to_dict(orient="records")

for impression in impressions:
    impression['impression_id'] = impression_id


In [ ]:
session_articles

In [ ]:
import os
import pandas as pd

class AnalystBehaviorSimulator:
    def __init__(self, agent):
        """
        Initialize the simulator with an LLM agent and tracking for analysts.

        Args:
            agent: The LLM agent configured to generate responses.
        """
        self.agent = agent
        self.analyst_data = {}  # Dictionary to store analyst-specific history and impressions
        self.history_file = "/app/datasets/history.tsv"  # Filepath for user history
        self.history_df = self.load_or_create_history()  # Load or create the history file

    def load_or_create_history(self):
        """
        Load the user history file if it exists, otherwise create an empty one.

        Returns:
            pd.DataFrame: The loaded or newly created history DataFrame.
        """
        if os.path.exists(self.history_file):
            print(f"Loading history from {self.history_file}")
            return pd.read_csv(self.history_file, sep="\t")
        else:
            print(f"History file not found. Creating {self.history_file}")
            # Create an empty DataFrame with appropriate columns
            df = pd.DataFrame(columns=["user_id", "history"])
            df.to_csv(self.history_file, sep="\t", index=False)
            return df

    def save_history(self):
        """
        Save the current history DataFrame back to the history.tsv file.
        """
        self.history_df.to_csv(self.history_file, sep="\t", index=False)
        print(f"History saved to {self.history_file}")

    def add_to_history(self, user_id, news_id):
        """
        Add a news article to the user's history.

        Args:
            user_id (str): The ID of the analyst.
            news_id (str): The ID of the news article.
        """
        # Check if the user already has a history entry
        if user_id in self.history_df["user_id"].values:
            # Append the new article to the existing history
            current_history = self.history_df.loc[self.history_df["user_id"] == user_id, "history"].values[0]
            updated_history = f"{current_history} {news_id}" if current_history else news_id
            self.history_df.loc[self.history_df["user_id"] == user_id, "history"] = updated_history
        else:
            # Add a new entry for the user
            new_entry = {"user_id": user_id, "history": news_id}
            self.history_df = pd.concat([self.history_df, pd.DataFrame([new_entry])], ignore_index=True)

        # Save the updated history
        self.save_history()


In [ ]:
analyst

In [ ]:
async def test_acting_as_person(agent, analyst, history, impression):
    gender = {"F":'female',"M":'male'}
    
    
    # Build the prompt
    prompt = f"""
    You are {analyst['name']}, a {analyst['age']}-year-old {analyst['gender']} working as a {analyst['job']}.
    {analyst['description']}

    Session Details:
    - User ID: {analyst['uid']}
    - Impression ID: {impression['impression_id']}

    You have previously interacted with the following articles:
    {', '.join(history)}

    The news article is titled: '{impression['title']}'
    - News ID: {impression['news_id']}

    Would you click on this article? (Respond with 1 for clicked, 0 for not clicked)
    Format this as string of article ID separated by aspace, where the ID is followed by '-1' if clicked "
        "or '-0' if not clicked (e.g., 'N12345-1 N23456-0').
    """
    
    print(f"Prompt \n{prompt}")
    # Query the LLM
    result = await agent.run(prompt)
    print("Acting as the Person Response:")
    print(result.data if result else "No response.")
    return result


async def test_acting_on_behalf_of_person(agent, analyst, history, impression):
    # Build the prompt
    prompt = f"""
    You are an expert judge of human character and news consumption behavior.

    Session Details:
    - User ID: {analyst['uid']}
    - Impression ID: {impression['impression_id']}

    Here is the user's profile:
    - Name: {analyst['name']}
    - Age: {analyst['age']}
    - Gender: {analyst['gender']}
    - Primary News Interest: {analyst['primary_news_interest']}
    - Secondary News Interest: {analyst['secondary_news_interest']}
    - Job: {analyst['job']}
    - Description: {analyst['description']}

    The user has previously interacted with the following articles:
    {', '.join(history)}

    Evaluate the following article:
    - Title: '{impression['title']}'
    - News ID: {impression['news_id']}

    Would they click on this article? (Respond with 1 for clicked, 0 for not clicked)
    What is your reasoning for clicking or not clicking on the article?
    """
    print(f"Prompt \n{prompt}")
    # Query the LLM
    result = await agent.run(prompt)
    print("Acting on Behalf of the Person Response:")
    print(result.data if result else "No response.")
    return result

In [ ]:
# Run both experiments
async def run_experiments():
    print("Experiment 1: Acting as the Person")
    return await test_acting_as_person(agent, analyst, history, impressions[0])

    #print("\nExperiment 2: Acting on Behalf of the Person")
    #await test_acting_on_behalf_of_person(agent, analyst, history, impressions[0])

# Run the experiments
result = await run_experiments()

In [ ]:
type(result)

In [ ]:
result

In [ ]:
print(result.data.dict)

In [ ]:
ab = result.data.model_dump()

In [ ]:
ab

---

In [ ]:
# Run the model without any training -- can training do better than no training at all?  

In [ ]:
print(model.run_eval(dev_news_file, dev_behaviors_file))

In [ ]:
dev_news_file

In [ ]:
model.fit(train_news_file, train_behaviors_file, dev_news_file, dev_behaviors_file)

In [ ]:
res_syn = model.run_eval(dev_news_file, dev_behaviors_file)
print(res_syn)
 

In [ ]:
# Record results for tests - ignore this cell
store_metadata("group_auc", res_syn['group_auc'])
store_metadata("mean_mrr", res_syn['mean_mrr'])
store_metadata("ndcg@5", res_syn['ndcg@5'])
store_metadata("ndcg@10", res_syn['ndcg@10'])

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

## Output Prediction File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

This should be some test data.  Only MINDLARGE as it.

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(dev_news_file, dev_behaviors_file)

In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## References

https://wuch15.github.io/paper/EMNLP2019-NRMS.pdf


https://github.com/recommenders-team/recommenders/tree/main

In [ ]:
# Calculate elapsed time
elapsed_time = time.time() - start_time

# Convert the elapsed time into hours, minutes, and seconds
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)

# Print the result in H:M:S format
print(f"Elapsed time: {int(hours)}:{int(minutes)}:{int(seconds)}")